In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
import cv2
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [27]:
# Load images
mask_path = "/kaggle/input/abdataset/dataset/with_mask"
no_mask_path = "/kaggle/input/abdataset/dataset/without_mask"

def load_images(path, label):
    data, labels = [], []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

        if img is None:  # Check if image is loaded properly
            print(f"Warning: Unable to read {file_path}")
            continue  # Skip this image

        img = cv2.resize(img, (64, 64))  # Resize for consistency
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        data.append(features)
        labels.append(label)

    return np.array(data), np.array(labels)



In [28]:
# Load both categories
X_mask, y_mask = load_images(mask_path, 1)
X_no_mask, y_no_mask = load_images(no_mask_path, 0)

# Combine datasets
X = np.vstack((X_mask, X_no_mask))
y = np.hstack((y_mask, y_no_mask))

# Split into training & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_svm = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {svm_accuracy:.2f}")


SVM Accuracy: 0.88


In [30]:
mlp_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500)
mlp_model.fit(X_train, y_train)

# Predict and evaluate
y_pred_mlp = mlp_model.predict(X_test)
mlp_accuracy = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Accuracy: {mlp_accuracy:.2f}")


MLP Accuracy: 0.92


In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
# Load images
def load_images_CNN(path, label):
    data, labels = [], []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        img = cv2.imread(file_path)
        if img is None:
            print(f"Warning: Skipping unreadable file {file_path}")
            continue
        img = cv2.resize(img, (64, 64))  # Resize for CNN input
        img = img / 255.0  # Normalize pixels to [0, 1]
        data.append(img)
        labels.append(label)
    return np.array(data), np.array(labels)


In [33]:
X_mask_CNN, y_mask_CNN = load_images_CNN(mask_path, 1)
X_no_mask_CNN, y_no_mask_CNN = load_images_CNN(no_mask_path, 0)

# Combine datasets
X_CNN = np.vstack((X_mask_CNN, X_no_mask_CNN))
y_CNN = np.hstack((y_mask_CNN, y_no_mask_CNN))

# Split into train (80%) and validation (20%)
X_train_CNN, X_test_CNN, y_train_CNN, y_test_CNN = train_test_split(X_CNN, y_CNN, test_size=0.2, random_state=42)


In [34]:
# Experiment with different hyperparameters
learning_rates = [0.001, 0.0001]
batch_sizes = [32, 64, 128]
optimizers = {'Adam': keras.optimizers.Adam, 'SGD': keras.optimizers.SGD}
activations = ['relu', 'leaky_relu', 'sigmoid']


In [35]:
for batch_size in batch_sizes:
    for activation in activations:
        for lr in learning_rates:
            for opt_name, opt_class in optimizers.items():
                print(f"Training with batch_size={batch_size}, activation={activation}, learning_rate={lr}, optimizer={opt_name}")
                
                model = keras.Sequential([
                    layers.Conv2D(32, (3,3), activation=activation, input_shape=(64, 64, 3)),
                    layers.MaxPooling2D(2,2),
                    layers.Conv2D(64, (3,3), activation=activation),
                    layers.MaxPooling2D(2,2),
                    layers.Conv2D(128, (3,3), activation=activation),
                    layers.MaxPooling2D(2,2),
                    layers.Flatten(),
                    layers.Dense(128, activation=activation),
                    layers.Dropout(0.5),  # Prevent overfitting
                    layers.Dense(1, activation="sigmoid")  # Binary classification
                ])

                # Compile the model
                optimizer = opt_class(learning_rate=lr)
                model.compile(optimizer=optimizer,
                              loss="binary_crossentropy",
                              metrics=["accuracy"])

                # Train the model
                history = model.fit(X_train_CNN, y_train_CNN, epochs=10, batch_size=batch_size,
                                    validation_data=(X_test_CNN, y_test_CNN))

                # Evaluate the model
                val_loss, val_acc = model.evaluate(X_test_CNN, y_test_CNN)
                print(f"Validation Accuracy: {val_acc:.2f}")

Training with batch_size=32, activation=relu, learning_rate=0.001, optimizer=Adam


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.7637 - loss: 0.4761 - val_accuracy: 0.8974 - val_loss: 0.2492
Epoch 2/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9093 - loss: 0.2655 - val_accuracy: 0.9011 - val_loss: 0.2553
Epoch 3/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9190 - loss: 0.2111 - val_accuracy: 0.9243 - val_loss: 0.1955
Epoch 4/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9381 - loss: 0.1627 - val_accuracy: 0.9487 - val_loss: 0.1412
Epoch 5/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9494 - loss: 0.1456 - val_accuracy: 0.9426 - val_loss: 0.1733
Epoch 6/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9650 - loss: 0.0906 - val_accuracy: 0.9451 - val_loss: 0.1478
Epoch 7/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9633 - loss: 0.0981 - val_accuracy: 0.9512 - val_loss: 0.1372
Epoch 8/10
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9732 - loss: 0.0823 - val_accuracy: 0